In [ ]:
import pandas as pd
import glob
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

In [ ]:
color_dict = {'ls5-ls7': '#66c2a5',
              'ls7-ls8': '#fc8d62'}


# Accuracy Stats

In [ ]:
def calc_all_stats(tp, fp, fn):
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2 * (precision * recall) / (precision + recall)
    iou =  tp / (tp + fp + fn)
    print('precision: {}, recall: {}, f1: {}, iou: {}'.format(
        precision, recall, f1, iou))
    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'iou': iou
    }

def read_process_stats(csv):
    df = pd.read_csv(csv)
    return df.sum(axis=0)

In [ ]:
# Valid pixel counts
all_df_list = []
for csv in sorted(glob.glob('./out/valid_counts/*.csv')):
    df = read_process_stats(csv)
    df['year'] = int(os.path.basename(csv)[4:8])
    df['sat'] = os.path.basename(csv)[:3]
    all_df_list.append(pd.DataFrame(df).T)


In [ ]:
valid_df = pd.concat(all_df_list, axis=0).set_index(['sat', 'year'])

In [ ]:

valid_df['valid_perc'] = 100*valid_df['valid']/ np.max(valid_df['valid'])

In [ ]:
single_color_dict = {'ls5': '#66c2a5',
              'ls7': '#fc8d62',
              'ls8': '#8da0cb'}

In [ ]:
valid_df.loc['ls5','valid_perc'].plot(color=single_color_dict['ls5'], lw=1.75, label='Landsat 5')
valid_df.loc['ls7','valid_perc'].plot(color=single_color_dict['ls7'], lw=1.75, label='Landsat 7')
valid_df.loc['ls8','valid_perc'].plot(color=single_color_dict['ls8'], lw=1.75, label='Landsat 8')
plt.xlabel('Year')
plt.legend()
plt.ylabel('% Valid Pixels')

In [ ]:
all_outs = []
for csv in sorted(glob.glob('./out/*_compare.csv')):
    print(csv)
    df = read_process_stats(csv)
    out_dict = calc_all_stats(df['tp'], df['fp'], df['fn'])
    out_dict['year'] = int(os.path.basename(csv)[:4])
    all_outs.append(out_dict)

In [ ]:
out_df = pd.DataFrame(all_outs).set_index('year')*100
out_df['satellite'] = 'ls5-ls7'
out_df.loc[2014:, 'satellite'] = 'ls7-ls8'

In [ ]:
out_df.groupby('satellite').mean()

In [ ]:

out_df.loc[2003:].groupby('satellite').mean()

In [ ]:
out_df.loc[:2019].groupby('satellite').mean()

In [ ]:
def plot_accuracy_stat(out_df, stat, ax=None, legend=True):
    out_df.loc[out_df['satellite']=='ls5-ls7', stat].plot(c=color_dict['ls5-ls7'], ax=ax, label='LS5-7')
    out_df.loc[2013:2017, stat].plot(c=color_dict['ls7-ls8'], ax=ax, label='LS7-8')
    out_df.loc[2017:, stat].plot(c=color_dict['ls7-ls8'], style='--', ax=ax, label='LS7-8 (Orbit Drift)')
    if ax is not None:
        ax.set_title(stat.capitalize())
        if legend:
            ax.legend()
        ax.set_ylim(30, 100)
        ax.set_xlabel('Year')
        ax.set_ylabel('Score')

In [ ]:

fig, axs = plt.subplots(2,2, figsize=(7,6))
for i, acc_stat in enumerate(['precision','recall', 'f1', 'iou']):
    if i == 0:
        include_legend=True
    else:
        include_legend=False
    plot_accuracy_stat(out_df, acc_stat, ax=axs.flatten()[i], legend=include_legend)
for i, label in enumerate(['$(a)$', '$(b)$', '$(c)$', '$(d)$']):
    axs.flatten()[i].annotate(
            label,
            xy=(0, 1), xycoords='axes fraction',
            xytext=(0.2, -1.3), textcoords='offset fontsize',
            fontsize=12, verticalalignment='bottom', fontfamily='serif')
fig.tight_layout()

# Count, Area, and Overlap comparisons

In [ ]:
def convert_to_array(string):
    ar = np.fromstring(string.strip('[]'),sep=' ').astype(int)
    return ar[ar!=0]

def calc_overlap_stats(df1, df2, sat_comp, y):
    df1_overlap = df1.loc[df1.overlap_count>0]
    df2_overlap = df2.loc[df2.overlap_count>0]

    df1_no_overlap = df1.loc[df1.overlap_count==0]
    df2_no_overlap = df2.loc[df2.overlap_count==0]
    out_dict = {
        'year': y,
        'satellites': sat_comp,
        # Totals
        'sat1_total_area': df1['size'].sum(),
        'sat2_total_area': df2['size'].sum(),
        'sat1_mean_size': df1['size'].mean(),
        'sat2_mean_size': df2['size'].mean(),
        'sat1_count': df1['size'].shape[0],
        'sat2_count': df2['size'].shape[0],
        # Overlap
        'sat1_overlap_total_area': df1_overlap['size'].sum(),
        'sat2_overlap_total_area': df2_overlap['size'].sum(),
        'sat1_overlap_mean_size': df1_overlap['size'].mean(),
        'sat2_overlap_mean_size': df2_overlap['size'].mean(),
        'sat1_overlap_count': df1_overlap['size'].shape[0],
        'sat2_overlap_count': df2_overlap['size'].shape[0],
        # No Overlap
        'sat1_no_overlap_total_area': df1_no_overlap['size'].sum(),
        'sat2_no_overlap_total_area': df2_no_overlap['size'].sum(),
        'sat1_no_overlap_mean_size': df1_no_overlap['size'].mean(),
        'sat2_no_overlap_mean_size': df2_no_overlap['size'].mean(),
        'sat1_no_overlap_count': df1_no_overlap['size'].shape[0],
        'sat2_no_overlap_count': df2_no_overlap['size'].shape[0],
    }
    return out_dict

In [ ]:
all_outs = []
for y in range(2000, 2024):
    csv_list = glob.glob('./out/*{}*_overlaps.csv'.format(y))

    if len(csv_list) == 2:
        csv_list.sort()
        sat1_name = os.path.basename(csv_list[0])[:3]
        sat2_name = os.path.basename(csv_list[1])[:3]
        df1 = pd.read_csv(csv_list[0], converters={'overlaps':convert_to_array})
        df2 = pd.read_csv(csv_list[1], converters={'overlaps':convert_to_array})
        df1['overlap_count'] = df1['overlaps'].apply(len)
        df2['overlap_count'] = df2['overlaps'].apply(len)
        out_dict = calc_overlap_stats(df1, df2, sat_comp='{}-{}'.format(sat1_name, sat2_name), y=y)
        all_outs.append(out_dict)

In [ ]:
pd.DataFrame(all_outs).to_csv('./out/overlap_summaries.csv', index=False)

In [ ]:
overlap_stats = pd.read_csv('./out/overlap_summaries.csv').set_index('year')
# Remove after 2020, ls7 degradation
overlap_stats = overlap_stats.loc[:2019]

In [ ]:
# Full figure
fig, axs = plt.subplots(1,2, figsize=(10,5))
comparison_dict = {
    'sat1': 'Old Total',
    'sat2': 'Total',
    'sat2_overlap': 'Overlap',
    'sat2_no_overlap': 'No Overlap'
}
comparison_color_dict = {
    'sat1': 'red',
    'sat2': 'blue',
    'sat2_overlap': 'purple',
    'sat2_no_overlap': 'green'
}
lstyle_dict = {
    'ls5-ls7': '-',
    'ls7-ls8': '--'
}
for sat_comp in ['ls5-ls7', 'ls7-ls8']:
    for comparison in ['sat1', 'sat2','sat2_overlap', 'sat2_no_overlap']:
        comp_name = sat_comp[-3:] + ' ' + comparison_dict[comparison]

        overlap_stats.loc[overlap_stats['satellites']==sat_comp, 
                          '{}_total_area'.format(comparison)
                    ].plot(ax=axs[1],
                           label=comp_name,
                           color=comparison_color_dict[comparison],
                           linestyle=lstyle_dict[sat_comp])
        overlap_stats.loc[overlap_stats['satellites']==sat_comp, 
                          '{}_count'.format(comparison)
                    ].plot(ax=axs[0],
                           label=comp_name,
                           color=comparison_color_dict[comparison],
                           linestyle=lstyle_dict[sat_comp])
        axs[1].legend()

    axs[0].set_title('Count')
    axs[1].set_title('Area')
    axs[0].set_xlim(1999, 2021)
    axs[1].set_xlim(1999, 2021)


In [ ]:

# Full figure
fig, axs = plt.subplots(1,2, figsize=(10,5))
comparison_dict = {
    'sat1': 'Old Total',
    'sat2': 'Total',
    'sat2_overlap': 'Overlap',
    'sat2_no_overlap': 'No Overlap'
}
comparison_color_dict = {
    'sat1': 'red',
    'sat2': 'blue',
    'sat2_overlap': 'purple',
    'sat2_no_overlap': 'green'
}
lstyle_dict = {
    'ls5-ls7': '-',
    'ls7-ls8': '--'
}
for sat_comp in ['ls7-ls8']:
    for comparison in ['sat1', 'sat2','sat2_overlap', 'sat2_no_overlap']:
        comp_name = sat_comp[-3:] + ' ' + comparison_dict[comparison]

        overlap_stats.loc[overlap_stats['satellites']==sat_comp, 
                          '{}_total_area'.format(comparison)
                    ].plot(ax=axs[1],
                           label=comp_name,
                           color=comparison_color_dict[comparison],
                           linestyle=lstyle_dict[sat_comp])
        overlap_stats.loc[overlap_stats['satellites']==sat_comp, 
                          '{}_count'.format(comparison)
                    ].plot(ax=axs[0],
                           label=comp_name,
                           color=comparison_color_dict[comparison],
                           linestyle=lstyle_dict[sat_comp])
        axs[1].legend()

    axs[0].set_title('Count')
    axs[1].set_title('Area')
    axs[0].set_xlim(2013, 2021)
    axs[1].set_xlim(2013, 2021)


In [ ]:
overlap_stats_ls7_ls8 = overlap_stats.loc[2014:]

In [ ]:
overlap_stats_ls7 = overlap_stats_ls7_ls8[[col for col in overlap_stats_ls7_ls8.columns if col[:4]=='sat1']]
overlap_stats_ls8 = overlap_stats_ls7_ls8[[col for col in overlap_stats_ls7_ls8.columns if col[:4]=='sat2']]
overlap_stats_ls7.columns = [col[5:] for col in overlap_stats_ls7.columns]
overlap_stats_ls8.columns = [col[5:] for col in overlap_stats_ls8.columns]
overlap_stats_ls8['sat'] = 'LS8'
overlap_stats_ls7['sat'] = 'LS7'

In [ ]:
overlap_stats_both = pd.concat([overlap_stats_ls7, overlap_stats_ls8]).reset_index()

In [ ]:
overlap_stats_both = overlap_stats_both.groupby('sat').mean().reset_index()

In [ ]:
hatch_dict = {
    'overlap': None,
    'no_overlap': 'xxx',
}
alpha_dict = {
    'overlap': 1.0,
    'no_overlap': 0.5,
}
bar_color_dict = {'LS5': '#66c2a5',
              'LS7': '#fc8d62',
              'LS8': '#8da0cb'}

In [ ]:
labels=overlap_stats_both['year'].drop_duplicates()  # set the dates as labels
x0 = np.arange(len(labels))  # create an array of values for the ticks that can perform arithmetic with width (w)


# build the plots
sats = ['LS7','LS8']
stacks = len(sats)  # how many stacks in each group for a tick location

# set the width
w = 0.35

# this needs to be adjusted based on the number of stacks; each location needs to be split into the proper number of locations
x1 = [x0 - w/stacks, x0 + w/stacks]

fig, axs = plt.subplots(1,2, figsize=(8,4))
comps = ['overlap','no_overlap']
for x, sat in zip(x1, sats):
    bottom_count = 0
    bottom_area = 0
    for comparison in comps:
        height_count = overlap_stats_both.loc[overlap_stats_both['sat']==sat, comparison + '_count'].values / 1000
        axs[0].bar(x=x, height=height_count, width=w, bottom=bottom_count,
               hatch=hatch_dict[comparison], 
               color=bar_color_dict[sat],
               alpha=alpha_dict[comparison],
               label=sat + ' ' + comparison)
        bottom_count += height_count

        height_area = overlap_stats_both.loc[overlap_stats_both['sat']==sat, comparison + '_total_area'].values / 10000
        axs[1].bar(x=x, height=height_area, width=w, bottom=bottom_area,
               hatch=hatch_dict[comparison], 
               color=bar_color_dict[sat],
               alpha=alpha_dict[comparison],
               label=sat + ' ' + comparison)
        bottom_area += height_area
for ax in axs:
       ax.set_xticks([])
       _ = ax.set_xticklabels('')
legend_patches = [
    Patch(facecolor=bar_color_dict['LS7'], label='Landsat 7'),
    Patch(facecolor=bar_color_dict['LS8'],  label='Landsat 8'),
    Patch(facecolor='grey',alpha=0.2, label='Shared Reservoirs'),
    Patch(facecolor='grey',alpha=0.2, hatch='xxxx', label='Unshared Reservoirs')
]
axs[1].legend(handles=legend_patches)
axs[0].set_ylabel('Count (thousands)')
axs[1].set_ylabel('Area ($km^2$)')
axs[0].text(
    x=-0.375, y=axs[0].get_ylim()[1]*0.985,  # Relative position in axes coordinates (0,0 is bottom-left; 1,1 is top-right)
    s='(a)',          # The label text
    fontsize=10,
    fontstyle='italic',      # Font size
    verticalalignment='top',  # Align the text to the top
    horizontalalignment='left'  # Align the text to the left
)
axs[1].text(
    x=-0.375, y=axs[1].get_ylim()[1]*0.985,  # Relative position in axes coordinates (0,0 is bottom-left; 1,1 is top-right)
    s='(b)',          # The label text
    fontsize=10,      # Font size
    fontstyle='italic',      # Font size
    verticalalignment='top',  # Align the text to the top
    horizontalalignment='left'  # Align the text to the left
)
fig.tight_layout()